In [1]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt
import random

2023-09-28 11:49:08.000907: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-28 11:49:08.077135: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-28 11:49:08.078902: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-28 11:49:09.741119: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [21]:
# za treniranje nasih mreza koristicemo PokerHand data set sa Kaggle-a koji predstavlja karte koje neki 
# igraci imaju na raspolaganju (2 u ruci 3 na talonu) i na osnovu toga nas zadatak je da odredimo koju
# kombinaciju ima svaki igrac na osnovu trening skupa koji nam implicitno govori pravila
import pandas as pd

# ucitavamo podatke za trening pomocu pandas biblioteke 
data = pd.read_csv('treningPoker.csv')
train_set = data.iloc[:,:10]
labels = data.iloc[:,10:]

train_labels_cat = keras.utils.to_categorical(labels) 

class_names = ['Nothing','One pair','Two pairs','Three of a kind','Straight','Flush',
               'Full house','Four of a kind','Straight flush','Royal flush']

print(train_set.shape,labels.shape,train_labels_cat.shape)

(25010, 10) (25010, 1) (25010, 10)


In [28]:
# objekti klase Individual ce bit nase jedinke u GA
class Individual:
    def __init__(self,minLayers,maxLayers,minNodes,maxNodes,input_shape):
        self.input_shape = input_shape
        self.numLayers = random.randrange(minLayers,maxLayers)
        self.layers = [random.randrange(minNodes,maxNodes) for _ in range(self.numLayers)]
        self.model = self.create_model()
        self.acc = self.calc_acc()

    def create_model(self):
        modelList = [keras.layers.Flatten(input_shape=self.input_shape)]
        
        for i in range(self.numLayers):
            modelList.append(keras.layers.Dense(self.layers[i], activation='relu'))
        
        modelList.append(keras.layers.Dense(10, activation='softmax'))
        
        model = Sequential(modelList)
        model.compile(optimizer='adam',loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'])
        
        
        return model
    
    # fitnes funkcija neseg GA ce biti 'accuracy' posednje epohe treninga
    def calc_acc(self):
        history = self.model.fit(train_set, train_labels_cat, epochs=15)
        return history.history['accuracy'][-1]
    
    def __lt__(self,other):
        return self.acc < other.acc

In [29]:
def selection(population, tour_size):
    # odredjujemo koje ce se jedinke takmiciti da ostave potomstvo pomocu turnirske selekcije 
    competitors_idx = random.sample(range(len(population)), tour_size)  
    max_acc = -1.0
    max_idx = -1
    for i in range(tour_size):
        if population[competitors_idx[i]].acc > max_acc:
            max_acc = population[competitors_idx[i]].acc
            max_idx = competitors_idx[i]
    return max_idx

In [30]:
def crossover(individual_1, individual_2, child_1, child_2):
    # ukrstamo dve jedinke koristeci jednopoziciono ukrstanje
    crossing_point = random.randrange(0,min(individual_1.numLayers,individual_2.numLayers))
    
    child_1.layers[:crossing_point] = individual_1.layers[:crossing_point]
    child_1.layers[crossing_point:] = individual_2.layers[crossing_point:]
    child_1.numLayers = len(child_1.layers)
    
    child_2.layers[:crossing_point] = individual_2.layers[:crossing_point]
    child_2.layers[crossing_point:] = individual_1.layers[crossing_point:]
    child_2.numLayers = len(child_2.layers)
    

In [31]:
def mutation(individual,mutation_prob):
    # mutiramo tj. po odredjenoj verovatnoci menjamo svaki sloj jedne jedinke 
    for i in range(len(individual.layers)):
        if random.random() < mutation_prob:
            print("MUTACIJA")
            individual.layers[i] -= 1 

In [32]:
def ga(pop_size,num_iters,elitism,mutation_prob,tour_size,minLayers,maxLayers,minNodes,maxNodes,input_shape):
    
    # inicijalizujemo popuaciju po datim parametrima
    population = [Individual(minLayers,maxLayers,minNodes,maxNodes,input_shape) for _ in range(pop_size)]
    pocetni = max(population)
    new_population = population[:]

    # ovde cemo cuvati najbolje jedinke u svakoj generaciji
    best_in_gens = []
    
    for i in range(num_iters):
        print()
        print(f"iter_{i}")
        print()
        print([(population[i].numLayers, population[i].layers) for i in range(pop_size)])

        # uvodimo eltizam tj. odredjen procenat najboljih jedinki prezivi svaku generaciju tj. ostavi potomstvo 
        population.sort(key=lambda x: x.acc, reverse=True)
        new_population[:elitism] = population[:elitism]

        for i in range(elitism,pop_size,2):

            parent1_idx = selection(population, tour_size)
            parent2_idx = selection(population, tour_size)

            while parent1_idx == parent2_idx:
                parent2_idx = selection(population, tour_size)

            # ako su im roditelji isti nema potrebe za crossoverom
            if population[parent1_idx].layers == population[parent2_idx].layers:
                new_population[i] = population[parent1_idx]
                new_population[i+1] = population[parent1_idx]
                continue

            crossover(population[parent1_idx], population[parent2_idx], new_population[i], new_population[i+1])

            mutation(new_population[i], mutation_prob)
            mutation(new_population[i+1], mutation_prob)
            
            # ispisujemo novodobijene ukrstene jedinke 
            print(new_population[i].layers, new_population[i+1].layers)
            new_population[i].model = new_population[i].create_model()
            new_population[i+1].model = new_population[i+1].create_model()

            new_population[i].acc = new_population[i].calc_acc()
            new_population[i+1].acc = new_population[i+1].calc_acc()

        population[:] = new_population[:]
        best_in_gens.append(max(population))
    return best_in_gens
    

In [33]:
best_individuals = ga(pop_size=32,num_iters=5,elitism=2,mutation_prob=0.05,tour_size=8,
                     minLayers=4,maxLayers=10,minNodes=10,maxNodes=32,input_shape=(10,))
print("Na pocetku najbolji:", best_individuals[0].layers, best_individuals[0].acc)
print("Na kraju najbolji:", best_individuals[-1].layers, best_individuals[-1].acc)

#ucitavamo podatke za test
data = pd.read_csv('poker-hand-testing.csv')

test_set = data.iloc[:,:10]
labels_test = data.iloc[:,10:]

train_labels_cat1 = keras.utils.to_categorical(labels_test) 
print(test_set.shape,labels_test.shape,train_labels_cat1.shape)

# evaluiramo nas model za najboljeg iz prve
test_loss, test_acc = best_individuals[0].model.evaluate(test_set,train_labels_cat1,verbose=2)
print("konacno_prvi: ",test_acc)

# i najboljeg iz poslednje generacije
test_loss, test_acc = best_individuals[-1].model.evaluate(test_set,train_labels_cat1,verbose=2)
print("konacno_poslednji: ",test_acc)

Epoch 1/15
782/782 [==============================] - 4s 3ms/step - loss: 1.0365 - accuracy: 0.4830
Epoch 2/15
782/782 [==============================] - 3s 3ms/step - loss: 0.9782 - accuracy: 0.5060
Epoch 3/15
782/782 [==============================] - 3s 3ms/step - loss: 0.9654 - accuracy: 0.5284
Epoch 4/15
782/782 [==============================] - 3s 3ms/step - loss: 0.9563 - accuracy: 0.5419
Epoch 5/15
782/782 [==============================] - 3s 3ms/step - loss: 0.9483 - accuracy: 0.5480
Epoch 6/15
782/782 [==============================] - 3s 3ms/step - loss: 0.9418 - accuracy: 0.5528
Epoch 7/15
782/782 [==============================] - 3s 3ms/step - loss: 0.9380 - accuracy: 0.5560
Epoch 8/15
782/782 [==============================] - 3s 3ms/step - loss: 0.9336 - accuracy: 0.5581
Epoch 9/15
782/782 [==============================] - 3s 3ms/step - loss: 0.9301 - accuracy: 0.5616
Epoch 10/15
782/782 [==============================] - 3s 3ms/step - loss: 0.9241 - accuracy: 0.5671

782/782 [==============================] - 3s 4ms/step - loss: 0.9041 - accuracy: 0.5847
Epoch 14/15
782/782 [==============================] - 3s 4ms/step - loss: 0.8994 - accuracy: 0.5857
Epoch 15/15
782/782 [==============================] - 3s 3ms/step - loss: 0.8947 - accuracy: 0.5913
Epoch 1/15
782/782 [==============================] - 4s 3ms/step - loss: 1.0195 - accuracy: 0.4971
Epoch 2/15
782/782 [==============================] - 3s 3ms/step - loss: 0.9734 - accuracy: 0.5176
Epoch 3/15
782/782 [==============================] - 3s 3ms/step - loss: 0.9605 - accuracy: 0.5398
Epoch 4/15
782/782 [==============================] - 3s 3ms/step - loss: 0.9539 - accuracy: 0.5447
Epoch 5/15
782/782 [==============================] - 3s 3ms/step - loss: 0.9472 - accuracy: 0.5497
Epoch 6/15
782/782 [==============================] - 3s 3ms/step - loss: 0.9421 - accuracy: 0.5569
Epoch 7/15
782/782 [==============================] - 3s 3ms/step - loss: 0.9358 - accuracy: 0.5585
Epoch 8/1

782/782 [==============================] - 3s 3ms/step - loss: 0.9096 - accuracy: 0.5808
Epoch 11/15
782/782 [==============================] - 3s 3ms/step - loss: 0.9030 - accuracy: 0.5837
Epoch 12/15
782/782 [==============================] - 3s 3ms/step - loss: 0.8961 - accuracy: 0.5885
Epoch 13/15
782/782 [==============================] - 3s 3ms/step - loss: 0.8903 - accuracy: 0.5925
Epoch 14/15
782/782 [==============================] - 3s 3ms/step - loss: 0.8857 - accuracy: 0.5940
Epoch 15/15
782/782 [==============================] - 3s 3ms/step - loss: 0.8822 - accuracy: 0.5953
Epoch 1/15
782/782 [==============================] - 5s 4ms/step - loss: 1.0868 - accuracy: 0.4723
Epoch 2/15
782/782 [==============================] - 3s 4ms/step - loss: 0.9836 - accuracy: 0.4970
Epoch 3/15
782/782 [==============================] - 3s 4ms/step - loss: 0.9710 - accuracy: 0.5154
Epoch 4/15
782/782 [==============================] - 3s 4ms/step - loss: 0.9606 - accuracy: 0.5405
Epoch 

Epoch 1/15
782/782 [==============================] - 4s 3ms/step - loss: 1.0341 - accuracy: 0.4862
Epoch 2/15
782/782 [==============================] - 3s 3ms/step - loss: 0.9770 - accuracy: 0.5062
Epoch 3/15
782/782 [==============================] - 3s 3ms/step - loss: 0.9622 - accuracy: 0.5353
Epoch 4/15
782/782 [==============================] - 3s 3ms/step - loss: 0.9519 - accuracy: 0.5481
Epoch 5/15
782/782 [==============================] - 3s 3ms/step - loss: 0.9433 - accuracy: 0.5532
Epoch 6/15
782/782 [==============================] - 3s 3ms/step - loss: 0.9398 - accuracy: 0.5586
Epoch 7/15
782/782 [==============================] - 3s 3ms/step - loss: 0.9359 - accuracy: 0.5627
Epoch 8/15
782/782 [==============================] - 3s 3ms/step - loss: 0.9310 - accuracy: 0.5649
Epoch 9/15
782/782 [==============================] - 3s 3ms/step - loss: 0.9298 - accuracy: 0.5635
Epoch 10/15
782/782 [==============================] - 3s 3ms/step - loss: 0.9275 - accuracy: 0.5645

782/782 [==============================] - 3s 3ms/step - loss: 0.9254 - accuracy: 0.5622
Epoch 11/15
782/782 [==============================] - 3s 4ms/step - loss: 0.9222 - accuracy: 0.5672
Epoch 12/15
782/782 [==============================] - 3s 4ms/step - loss: 0.9172 - accuracy: 0.5705
Epoch 13/15
782/782 [==============================] - 3s 3ms/step - loss: 0.9138 - accuracy: 0.5759
Epoch 14/15
782/782 [==============================] - 3s 4ms/step - loss: 0.9117 - accuracy: 0.5758
Epoch 15/15
782/782 [==============================] - 3s 4ms/step - loss: 0.9059 - accuracy: 0.5803
Epoch 1/15
782/782 [==============================] - 5s 4ms/step - loss: 1.0298 - accuracy: 0.4900
Epoch 2/15
782/782 [==============================] - 3s 4ms/step - loss: 0.9691 - accuracy: 0.5248
Epoch 3/15
782/782 [==============================] - 3s 4ms/step - loss: 0.9603 - accuracy: 0.5367
Epoch 4/15
782/782 [==============================] - 3s 4ms/step - loss: 0.9503 - accuracy: 0.5491
Epoch 

...

782/782 [==============================] - 3s 4ms/step - loss: 0.9204 - accuracy: 0.5705
Epoch 11/15
782/782 [==============================] - 3s 4ms/step - loss: 0.9166 - accuracy: 0.5727
Epoch 12/15
782/782 [==============================] - 3s 4ms/step - loss: 0.9138 - accuracy: 0.5768
Epoch 13/15
782/782 [==============================] - 3s 4ms/step - loss: 0.9098 - accuracy: 0.5813
Epoch 14/15
782/782 [==============================] - 3s 4ms/step - loss: 0.9056 - accuracy: 0.5818
Epoch 15/15
782/782 [==============================] - 3s 4ms/step - loss: 0.8998 - accuracy: 0.5902
MUTACIJA
MUTACIJA
[24, 24, 29, 12, 24, 28, 28, 18] [24, 24, 29, 12, 24, 28, 28, 18]
Epoch 1/15
782/782 [==============================] - 5s 4ms/step - loss: 1.0451 - accuracy: 0.4912
Epoch 2/15
782/782 [==============================] - 3s 4ms/step - loss: 0.9721 - accuracy: 0.5243
Epoch 3/15
782/782 [==============================] - 3s 4ms/step - loss: 0.9597 - accuracy: 0.5368
Epoch 4/15
782/782 [==

2023-09-28 13:14:56.837369: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 40000000 exceeds 10% of free system memory.
2023-09-28 13:14:57.247325: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 80000000 exceeds 10% of free system memory.


31250/31250 - 55s - loss: 0.9248 - accuracy: 0.5638 - 55s/epoch - 2ms/step
konacno_prvi:  0.5638369917869568


2023-09-28 13:15:52.912696: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 40000000 exceeds 10% of free system memory.
2023-09-28 13:15:53.157435: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 80000000 exceeds 10% of free system memory.


31250/31250 - 58s - loss: 0.7871 - accuracy: 0.6568 - 58s/epoch - 2ms/step
konacno_poslednji:  0.6568220257759094


In [ ]:
#fashion_mnist = keras.datasets.fashion_mnist

#(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

#class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
 #              'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

#train_images = train_images / 255.0

#test_images = test_images / 255.0
#train_labels_cat = keras.utils.to_categorical(train_labels)
#test_labels_cat = keras.utils.to_categorical(test_labels)
#print(train_images.shape)